# A family tree - Exercise

Now that we are more familiar with logic programming, let's use it to solve an interesting
problem. Consider the following family tree:

<img src="./resources/tree.png"  style="height: 400px"/>

John and Megan have three sons – William, David, and Adam. The wives of William,
David, and Adam are Emma, Olivia, and Lily respectively. William and Emma have two
children – Chris and Stephanie. David and Olivia have five children – Wayne, Tiffany, Julie,
Neil, and Peter. Adam and Lily have one child – Sophia.

Based on these facts, can we create
a program that can tell us the name of Wayne's grandfather? Or tell us who Sophia's uncles are?

## 1. Facts

Define the two relations and the facts that represent the tree above:

In [1]:
from kanren import run, eq, var, conde, Relation, facts

father = Relation()
mother = Relation()

facts(father, ("John", "William"),
              ("John", "David"),
              ("John", "Adam"),
              ("William", "Chris"),
              ("William", "Stephanie"),
              ("David", "Wayne"),
              ("David", "Tiffany"),
              ("David", "Julie"),
              ("David", "Neil"),
              ("David", "Peter"),
              ("Adam", "Sophia"))

facts(mother, ("Megan", "William"),
              ("Megan", "David"),
              ("Megan", "Adam"),
              ("Emma", "Stephanie"),
              ("Emma", "Chris"),
              ("Olivia", "Tiffany"),
              ("Olivia", "Julie"),
              ("Olivia", "Neil"),
              ("Olivia", "Peter"),
              ("Lily", "Sophia"))

## 2. Rules

Write a rule for each of the cases below:

In [3]:
# Check if x is the parent of y => x is the father OR the mother of y
def parent(x, y):
    return conde([father(x,y)],[mother(x,y)])

# Check if x is the grandparent of y => x is the parent of the parent of y
def grandparent(x, y):
    z = var()
    return conde( (parent(x,z), parent(z,y)) )

# Check for sibling relationship between x and y => they have the same parent
def sibling(x, y):
    z = var()
    return conde((parent(z,x),parent(z,y)))

# Check if x is y's uncle => the father of x is the grandparent of y
def uncle(x, y):
    z = var()
    return conde((parent(z,x),grandparent(z,y)))


You can check your rules by executing the following code. Check in the family tree above if Kanren displays the right answers.

In [5]:
x = var()

print(run(0, x, parent(x, "Julie")))
print(run(0, x, grandparent("John", x)))
print(run(0, x, sibling(x, "Peter")))
print(run(0, x, uncle(x, "Peter")))

('David', 'Olivia')
('Chris', 'Sophia', 'Wayne', 'Stephanie', 'Neil', 'Julie', 'Peter', 'Tiffany')
('Peter', 'Wayne', 'Neil', 'Tiffany', 'Julie', 'Neil', 'Julie', 'Peter', 'Tiffany')
('Adam', 'Adam', 'David', 'William', 'William', 'David')


## 3. Ask Kanren

Now that our facts and rules are ready, we can ask Kanren anything we would like to know. Complete the code if necessary.

In [7]:
# John's children
name = "John"
output = run(0, x, parent("John", x)) # Remove the print statement here
print("\nList of %s's children:" % name)
for item in output:
    print(item)


List of John's children:
William
Adam
David


In [8]:
# William's mother
name = "William"
output = run(0, x, mother(x, "William")) # Remove the print statement here
print("\n %s's mother: %s\n" % (name, output[0]))


 William's mother: Megan



In [9]:
# Adam's parents
name = "Adam"
output = run(0, x, parent(x, "Adam")) # Remove the print statement here
print("\nList of %s's parents:" % name)
for item in output:
    print(item)


List of Adam's parents:
John
Megan


In [10]:
# Wayne's grandparents
name = "Wayne"
output = run(0, x, grandparent(x, "Wayne")) # Remove the print statement here
print("\nList of %s's grandparents:" % name)
for item in output:
    print(item)


List of Wayne's grandparents:
John
Megan


In [12]:
# Megan's grandchildren
name = "Megan"
output = run(0, x, grandparent("Megan", x)) # Remove the print statement here
print("\nList of %s's grandchildren:" % name)
for item in output:
    print(item)


List of Megan's grandchildren:
Wayne
Chris
Sophia
Neil
Stephanie
Julie
Peter
Tiffany


In [14]:
# Neil's siblings
name = "Neil"
output = run(0, x, sibling(x, "Neil")) # Remove the print statement here
siblings = [x for x in output if x != name] # you can't be a sibling of yourself
print("\nList of %s's siblings:" % name)
for item in siblings:
    print(item)


List of Neil's siblings:
Peter
Wayne
Tiffany
Julie
Julie
Peter
Tiffany


In [15]:
# Sophia's uncles
name = "Sophia"
name_father = run(0, x, father(x, name))[0]
output = run(0, x, uncle(x, "Sophia")) # Remove the print statement here
output = [x for x in output if x != name_father] # your father is not your uncle
print("\nList of %s's uncles:" % name)
for item in output:
    print(item)


List of Sophia's uncles:
David
William
William
David


In [16]:
# All spouses
a, b, c = var(), var(), var()
output = run(0, (a, b), father(a, c), mother(b, c))
print("\nList of all spouses:")
for item in output:
    print('Husband: %s <==> Wife: %s' % item)


List of all spouses:
Husband: William <==> Wife: Emma
Husband: John <==> Wife: Megan
Husband: David <==> Wife: Olivia
Husband: Adam <==> Wife: Lily
Husband: John <==> Wife: Megan
Husband: John <==> Wife: Megan
Husband: David <==> Wife: Olivia
Husband: David <==> Wife: Olivia
Husband: David <==> Wife: Olivia
Husband: William <==> Wife: Emma
